In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
# Add modules to the path
import sys
sys.path.append('C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis')
import matplotlib.pyplot as plt
from hernando_behavioural_analysis_dev.BehaviouralAnalysis.utils import custom_functions as cuf
import matplotlib.cm as cm
import numpy as np
import os
import matplotlib
import pandas as pd
import pickle
from utils.post_processing_utils import *
import seaborn as sns

In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
def moving_average(x, window=50):
    return np.convolve(x, np.ones(window), 'valid') / window

In [5]:
processed_data_dir = os.path.join('W:\\photometry_2AC\\processed_data\\state_change_data')
state_change_data_file = os.path.join(processed_data_dir, 'value_switch_nacc_mice.csv')
pre_pc = []
post_pc = []
mice = [ 'SNL_photo28', 'SNL_photo30', 'SNL_photo31', 'SNL_photo32', 'SNL_photo34', 'SNL_photo35']
mouse_ids = []
session_ids = []
moving_avs = []
reward_blocks = []
#mice = ['SNL_photo28', 'SNL_photo30', 'SNL_photo31', 'SNL_photo32', 'SNL_photo33', 'SNL_photo34', 'SNL_photo35']
for mouse_num, mouse_id in enumerate(mice):
    all_experiments = get_all_experimental_records()
    sessions = all_experiments[(all_experiments['mouse_id'] == mouse_id) & (all_experiments['experiment_notes'] == 'value switch')]['date'].values
    for date in sessions:
        experiment_to_process = all_experiments[(all_experiments['date'] == date) & (all_experiments['mouse_id'] == mouse_id)]
        traces, trial_data = open_experiment(experiment_to_process)
        trial_data.loc[trial_data['Trial outcome'] == 3, 'Trial outcome'] = 0
        red_trial_data = trial_data[trial_data['State name'] == 'TrialStart']
        post_trials = red_trial_data[red_trial_data['Trial num'] >= 100]
        pre_trials = red_trial_data[red_trial_data['Trial num'] < 100]
        post_reward_side = post_trials['Reward block'].unique()[0]
        if post_reward_side == 1:
            side = -1
        elif post_reward_side == 5:
            side = 1

        pre_bias = cuf.RBias(pre_trials['First response'], pre_trials['First choice correct'])* side
        post_bias = cuf.RBias(post_trials['First response'], post_trials['First choice correct']) *side
        post_pc.append(post_bias)
        pre_pc.append(pre_bias)
        session_ids.append(date)
        mouse_ids.append(mouse_id)
        reward_blocks.append(post_reward_side)
        
        


In [6]:
behavioural_change = {}
behavioural_change['mouse'] = mouse_ids
behavioural_change['session'] = session_ids
behavioural_change['pre bias'] = pre_pc
behavioural_change['post bias'] = post_pc
behavioural_change_df = pd.DataFrame(behavioural_change)

In [8]:
df_for_plot = behavioural_change_df.groupby(['mouse'])[['pre bias', 'post bias']].apply(np.mean)
df_for_plot = df_for_plot.reset_index()

In [9]:
df_for_plot1 = df_for_plot.set_index('mouse').transpose()


In [10]:
def francescas_plot(ax, data, show_err_bar=False, mean_linewidth=4, mean_line_color='blue', legend=False):
    """Plot lines for all mice from early to late.
    """
    data.plot(ax=ax, color='gray', legend=False)
    data.mean(1).plot(ax=ax,linewidth=mean_linewidth,color=mean_line_color)

    if show_err_bar:
        yerr = data.std(axis=1)

        plt.errorbar(np.array([0,1]), data.mean(1), yerr, color=mean_line_color, linewidth=4)

    # drop the left and bottom spine for cool looking effect
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

In [13]:
fig, ax = plt.subplots(figsize=[3.5,4])
francescas_plot(ax, df_for_plot1, mean_line_color='#7FB5B5', mean_linewidth=6, show_err_bar=False)
plt.xticks([0, 1], ['equal rewards', 'unequal rewards'], fontsize=12)
plt.ylabel('Bias towards BIG side', fontsize=12)
plt.tight_layout()


In [14]:
df_for_plot

,mouse,pre bias,post bias
0,SNL_photo28,0.036667,0.080400
1,SNL_photo30,-0.002000,0.047888
2,SNL_photo31,-0.011667,0.109097
3,SNL_photo32,-0.033333,0.037873
4,SNL_photo34,-0.001667,0.033425
5,SNL_photo35,-0.005000,0.049377
